In [1]:
%env CONDA_PREFIX

'C:\\Users\\ai41\\anaconda3\\envs\\tensorflow'

In [82]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os

In [83]:
import cv2   #opencv
import imutils

In [84]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [132]:
#IMG_DIR="d:/Ai/project1/imgs/train"
IMG_DIR="d:/Ai/project1/"
df = pd.read_csv(IMG_DIR+"driver_imgs_list.csv")
df['img'] = IMG_DIR + 'imgs/train/' + df['classname'] + '/' + df['img']
x = df["classname"]
y = df["img"]
df = df.sample(frac=1)
print(type(df))
print(type(x))
df.head(20)
df_train = df[:14000]
df_test = df[14000:]

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [133]:
df_train

,subject,classname,img
577,p002,c7,d:/Ai/project1/imgs/train/c7/img_46523.jpg
21253,p075,c5,d:/Ai/project1/imgs/train/c5/img_60850.jpg
16452,p052,c2,d:/Ai/project1/imgs/train/c2/img_6450.jpg
14348,p049,c8,d:/Ai/project1/imgs/train/c8/img_42327.jpg
10951,p041,c2,d:/Ai/project1/imgs/train/c2/img_97201.jpg
...,...,...,...
8465,p026,c3,d:/Ai/project1/imgs/train/c3/img_85671.jpg
10494,p039,c5,d:/Ai/project1/imgs/train/c5/img_3400.jpg
1962,p014,c4,d:/Ai/project1/imgs/train/c4/img_27095.jpg
17879,p061,c1,d:/Ai/project1/imgs/train/c1/img_357.jpg


In [104]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [134]:
datagen = ImageDataGenerator(rescale=1./255,                  #이미지 데이터 /255 : Minmaxscaling
                                      zoom_range=0.1,    #원래 영상에 +-20%(0.8~1.2) 정도 크기를 재조정해라
                                      width_shift_range=0.1, #좌우로 약간씩 이동한 영상으로 만들라(모든 사진이 정가운데x라서)
                                      height_shift_range=0.1,   #위아래로 약간씩(+-20%) 이동한 영상 
                                      validation_split=0.2   #rotation: 각도도 범위 지정 가능 #학습 data 80%, 테스트 데이터 20% 변형된 데이터가 나올 수 있게 해서 학습
                                     )   #데이터 증강 
train_data = datagen.flow_from_dataframe(df_train,
                                         x_col='img',
                                         y_col = 'classname',
                                        target_size=(123,155),  #타겟 사이즈
                                        batch_size=16,      #배치 사이즈 32장 추출
                                        class_mode = 'categorical',   #출력 노드가 one-hot encoding
                                       subset = 'training')   #트레이닝용 80%

valid_data = datagen.flow_from_dataframe(df_train,
                                         x_col='img',
                                         y_col = 'classname',
                                         target_size=(123,155),
                                     batch_size=16,
                                     class_mode = 'categorical',   #one-hot encoding
                                     subset = 'validation')   #validation용
#원핫 인코딩을 위해 클래스 수를 보니 5개더라

datagen = ImageDataGenerator(rescale=1./255,                  #이미지 데이터 /255 : Minmaxscaling
                                      zoom_range=0.1,    #원래 영상에 +-20%(0.8~1.2) 정도 크기를 재조정해라
                                      width_shift_range=0.1, #좌우로 약간씩 이동한 영상으로 만들라(모든 사진이 정가운데x라서)
                                      height_shift_range=0.1,   #위아래로 약간씩(+-20%) 이동한 영상 
#                                       validation_split=0.2   #rotation: 각도도 범위 지정 가능 #학습 data 80%, 테스트 데이터 20% 변형된 데이터가 나올 수 있게 해서 학습
                                     )   #데이터 증강 
test_data = datagen.flow_from_dataframe(df_test,
                                         x_col='img',
                                         y_col = 'classname',
                                        target_size=(123,155),  #타겟 사이즈
                                        batch_size=16,      #배치 사이즈 32장 추출
                                        class_mode = 'categorical',   #출력 노드가 one-hot encoding
                                       subset = 'training')   #트레이닝용 80%

Found 11200 validated image filenames belonging to 10 classes.
Found 2800 validated image filenames belonging to 10 classes.
Found 8424 validated image filenames belonging to 10 classes.


In [107]:
IMG_DIR="d:/Ai/project1/imgs/train/"

In [136]:
seed=0
np.random.seed(seed)
tf.compat.v1.set_random_seed(seed)

model = Sequential()
#conv1
model.add(Conv2D(32, (5, 5), 2, input_shape=(123,155,3),      
                activation="relu"))
#maxpooling 1
model.add(MaxPooling2D(pool_size=(2,2)))
#conv2
model.add(Conv2D(64, (3,3), activation="relu"))
#maxp 2
model.add(MaxPooling2D(pool_size=(2,2)))  #conv1>mp1>conv2>mp2>flatten해서>fc1>fc2
model.add(Dropout(0.25))   
#탈락율이 25% : 다 하면 너무 학습 패턴에 맞추어진다. (과적합방지)
#어떤 뉴런은 탈락시킨다 = 이 뉴런은 수정안하고 나머지 75%는 수정, 선택되는 뉴런은 할때마다 달라진다

model.add(Conv2D(128,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))   

model.add(Flatten())
model.add(Dense(units=30, activation="relu")) #은닉층 1 :relu,입력 17개, 뉴런 30개 
model.add(Dense(units=15, activation="relu"))
model.add(Dense(units=10, activation="softmax"))

model.compile(loss="categorical_crossentropy",  #spares는 one-hot coding 안할때
             optimizer="adam",
             metrics=["accuracy"])
#모델 저장 폴더 설정
MODEL_DIR = "d:/Ai/project1/"
if not os.path.exists (MODEL_DIR):
    os.mkdir(MODEL_DIR)                          #().mkdir :폴더 생성
                                                                    
#모델 저장 조건 설정            #파일이름에는 빈칸이 들어가면 안된다
modelfile = MODEL_DIR+'CNN_project1_best.h5'   
checkpointer = ModelCheckpoint(filepath=modelfile,      
                              moniter="val_loss", verbose=1,
                              save_best_only=True) 

#학습의 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=10)


#모델 학습 및 저장  
fit_history = model.fit(train_data, 
                        epochs=50, verbose=1, 
                        validation_data=valid_data,
                        callbacks=[early_stopping_callback, checkpointer])

Epoch 1/50
700/700 [==============================] - ETA: 0s - loss: 2.1822 - accuracy: 0.1611
Epoch 00001: val_loss improved from inf to 1.88811, saving model to d:/Ai/project1\CNN_project1_best.h5
700/700 [==============================] - 98s 140ms/step - loss: 2.1822 - accuracy: 0.1611 - val_loss: 1.8881 - val_accuracy: 0.2768
Epoch 2/50
700/700 [==============================] - ETA: 0s - loss: 1.6598 - accuracy: 0.3676
Epoch 00002: val_loss improved from 1.88811 to 1.29382, saving model to d:/Ai/project1\CNN_project1_best.h5
700/700 [==============================] - 100s 142ms/step - loss: 1.6598 - accuracy: 0.3676 - val_loss: 1.2938 - val_accuracy: 0.5000
Epoch 3/50
700/700 [==============================] - ETA: 0s - loss: 1.2377 - accuracy: 0.5204
Epoch 00003: val_loss improved from 1.29382 to 1.01827, saving model to d:/Ai/project1\CNN_project1_best.h5
700/700 [==============================] - 99s 142ms/step - loss: 1.2377 - accuracy: 0.5204 - val_loss: 1.0183 - val_accura

Epoch 26/50
700/700 [==============================] - ETA: 0s - loss: 0.1830 - accuracy: 0.9424
Epoch 00026: val_loss improved from 0.11290 to 0.10106, saving model to d:/Ai/project1\CNN_project1_best.h5
700/700 [==============================] - 98s 140ms/step - loss: 0.1830 - accuracy: 0.9424 - val_loss: 0.1011 - val_accuracy: 0.9736
Epoch 27/50
700/700 [==============================] - ETA: 0s - loss: 0.1927 - accuracy: 0.9371
Epoch 00027: val_loss did not improve from 0.10106
700/700 [==============================] - 100s 143ms/step - loss: 0.1927 - accuracy: 0.9371 - val_loss: 0.1058 - val_accuracy: 0.9707
Epoch 28/50
700/700 [==============================] - ETA: 0s - loss: 0.1776 - accuracy: 0.9437
Epoch 00028: val_loss did not improve from 0.10106
700/700 [==============================] - 100s 143ms/step - loss: 0.1776 - accuracy: 0.9437 - val_loss: 0.1089 - val_accuracy: 0.9668
Epoch 29/50
700/700 [==============================] - ETA: 0s - loss: 0.1833 - accuracy: 0.941